<a href="https://colab.research.google.com/github/AlekhSaxena/GPT/blob/main/Finance_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/')

In [ ]:
import pandas as pd
mc =pd.read_excel('MCAP31032023_0.xlsx')

In [ ]:
mc.shape
print(mc.head())
mc=mc.head(500)

In [ ]:
!pip install yfinance
!pip install tqdm

In [ ]:
import yfinance as yf
from tqdm import tqdm


In [ ]:
x=pd.DataFrame()
for sym in tqdm(mc.Symbol):
    try:
        nse=yf.download(f"{sym}.NS",period='1d',interval="5m")
        nse['sym']=sym
        nse=nse.reset_index()
        bse=yf.download(f"{sym}.BO",period='1d',interval="5m")
        bse['sym']=sym
        bse=bse.reset_index()
        final=nse.merge(bse,on=['Datetime','sym'],suffixes=('_NSE','_BSE'),how="inner")
    except Exception as e:
        print(f"Failed to download {sym} :{e}")
        continue
    x=pd.concat([x,final])

In [ ]:
x.to_csv('/content/drive/My Drive/finance_data_500.csv')

In [ ]:
x['diff_percentage']=(abs(x['Adj Close_NSE']-x['Adj Close_BSE']))/(x[['Adj Close_NSE','Adj Close_BSE']].min(axis=1))

In [ ]:
x['diff_percentage'].describe()

In [ ]:
x[(x['Volume_BSE'] > 10000) & (x['Volume_NSE'] > 10000)].sort_values(by='diff_percentage',ascending=False)

In [ ]:
x[(x['Volume_BSE'] > 1000) & (x['Volume_NSE'] > 1000)].sort_values(by='diff_percentage',ascending=False)